# Storage manager

There are two types of storage in tf project
- file storage buckets (google storage bucket)
- data warehouse (bigquery)

## Storage bucket

In [ ]:
bucket_name = "tmp"
!gsutil mb gs://{bucket_name}/
!gsutil rm -r gs://{bucket_name}/

In [ ]:
# GCP to GS
code_name = 'triangle_high_time_res_4M_fix'
!gsutil -m rsync -d -r models/{code_name} gs://tf_mirror/{code_name}

In [ ]:
# GS to local
code_name = 'triangle_high_time_res_4M_fix'
!mkdir models/{code_name}
!gsutil -m rsync -r gs://tf_mirror/{code_name} models/{code_name}

## Bigquery

In [ ]:
13.8*8